## Import

In [1]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
from selenium import webdriver
from tqdm import tqdm
from bs4 import BeautifulSoup
from IB_utils import *

import requests
import streamlit as st
import pandas as pd
import numpy as np
import OpenDartReader
import warnings
import time, datetime
import re, os

warnings.filterwarnings('ignore')

In [2]:
options = Options()
options.add_argument('--headless')

def get_driver(viz_opt = False):
    #return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    if viz_opt:
        return webdriver.Chrome()
    else:
        return webdriver.Chrome(options=options)

In [6]:
opt = 'IB전략'
start_dt = '2023-02-23'
end_dt = '2023-03-20'
start_dt2 = datetime.datetime.strftime(datetime.datetime.strptime(end_dt, '%Y-%m-%d') - datetime.timedelta(days = 80), '%Y-%m-%d')

dart_df, dart = initial_set(start_dt2, end_dt)

# kind
driver = get_driver(viz_opt = True)
driver.set_window_size(1920, 1080)

table = set_kind(driver, start_dt2, end_dt)
kind_output = get_kind_inner(driver, table)
first_df = post_proc(dart_df, kind_output, start_dt)

# ipo stock
driver = get_driver()
driver.set_window_size(1920, 1080)
ipo_df = ipo_main(driver, first_df)
first_df = pd.merge(first_df, ipo_df, on = 'corp_name', how = 'left')
first_df.replace(np.NaN, 0, inplace = True)
first_df['key'] = [change_join(x) if "스팩" in x else x for x in list(first_df.corp_name)]

outer_df = get_38(start_dt, end_dt)
second_df = pd.merge(first_df, outer_df, left_on = 'key', right_on = '기업명', how = 'inner')
del second_df['기업명'], second_df['key'], second_df['stock_code_x']
second_df.rename(columns = {'stock_code_y':'stock_code'}, inplace = True)

# DART
third_df = get_dd(dart, second_df)
third_df, fourth_df = get_d_tables(dart, third_df)
head_df = change_form(third_df, opt)

,corp_code,corp_name,stock_code,corp_cls,rcept_no,상장주식수,주요제품,상장주선인,공모가,경쟁률,신주모집,구주매출,기관배정수량,상장일,납입일,공모주식수,수요예측(시작일),수요예측(종료일),청약일
0,01351080,지아이이노베이션,358570,코스닥,20230320000170,22004200,면역항암제 및 알레르기 치료제,"하나증권주식회사,엔에이치투자증권주식회사,삼성증권㈜",13000,0,2000000,0,1500000,2023-03-09,2023-03-03,2000000,2023-02-21,2023-02-22,2023-02-27
1,00231901,LB인베스트먼트,309960,코스닥,20230316001463,23217239,"창업자에 대한 투자, 중소기업 창업투자자금의 관리",미래에셋증권 주식회사,5100,0,3078698,1539349,3463347,2023-03-29,2023-03-23,4618047,2023-03-13,2023-03-14,2023-03-20
2,01690235,유안타제12호스팩,446150,코스닥,20230310000641,4770000,금융 지원 서비스,유안타증권㈜,2000,0,4500000,0,3375000,2023-03-17,2023-03-10,4500000,2023-03-02,2023-03-03,2023-03-07
3,01683387,미래에셋드림스팩1호,442900,코스닥,20230309000435,7680000,인수합병,미래에셋증권 주식회사,10000,0,7000000,0,5250000,2023-03-15,2023-03-09,7000000,2023-02-27,2023-02-28,2023-03-06
4,00872656,금양그린파워,282720,코스닥,20230307000630,12119500,"전기공사, 토목건축공사 등",삼성증권㈜,10000,0,3007320,0,2255490,2023-03-13,2023-03-07,3007320,2023-02-23,2023-02-24,2023-03-02
